Parsing all the available source html with quest text fragments and transitions options from the site quest-book.ru. 

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm

In [14]:
test_page = 'https://quest-book.ru/online/?s=1'

In [42]:
def get_quests_links(menu_page_href):
    '''
    Collects all the relative links (quest identificators) from single quest-book.ru menu page
    '''
    page_soup = BeautifulSoup(urlopen(menu_page_href))
    return [quest_href['href'] for quest_href in page_soup.find_all('a', href=True, class_="view-game-btn")]

In [47]:
all_quest_hrefs = []
menu_prefix = 'https://quest-book.ru/online/?s='
start_index = 0
#Number of quest, which were available at the time of parsing (04.01.2021)
end_index = 180
step = 12

for menu_page_start_index in range(start_index, end_index, step):
    menu_page_href = menu_prefix + str(menu_page_start_index)
    all_quest_hrefs += get_quests_links(menu_page_href)

In [54]:
def get_xml_name(quest_href_part):
    '''
    Creates filename for the quest data using its relative link
    '''
    return quest_href_part.split('/')[-2] + '.xml'

In [84]:
failes = []

In [82]:
def download_quest_data(href_part, upload_dir):
    global failes
    #Route for get-query for the quest data
    full_href = 'https://quest-book.ru' + href_part + 'data'
    response = requests.get(full_href)
    #If the response status is success code, create a file in the upload_dir and write down the data
    if response.status_code == 200:
        xml_name = get_xml_name(href_part)
        file_path = os.path.join(upload_dir, xml_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response.text)
    #If response status is not success, add the relative link into failes list to check it later
    else:
        failes.append(href_part)

In [83]:
UPLOAD_DIR = 'D:\Диплом_текстовые_квесты\Data\quest-book'

In [88]:
for quest_href_part in tqdm(all_quest_hrefs):
    download_quest_data(quest_href_part, UPLOAD_DIR)

100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [01:20<00:00,  2.24it/s]


In [89]:
#No failes
len(failes)

0